TABU- HEURISTIC 1

In [1]:
import numpy as np
import random
import pandas as pd
import copy
from collections import defaultdict
from networkx import nx
from copy import copy
import time
import pickle

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import tree

from sklearn.ensemble import AdaBoostRegressor
import seaborn as sns

In [3]:
adjacency_df=pd.DataFrame(pd.read_csv("C:/Users/Lenovo/Desktop/adjacency_df-n100-m400.csv"))


In [4]:
n = len(adjacency_df)

adjacency_df.columns=(range(1,(n+4)))
adjacency_df=adjacency_df.rename(columns={n+1:'Threshold',n+2:'IsInfected', n+3:'IsRisky' },errors='raise')

adjacency_df.index=(range(1,(n+1)))

m=((adjacency_df.loc[:,range(1,n+1)]>0).sum().sum())/2


zeros=[1,0.3,0.1]
ones=[1,0.5,0.3]

In [5]:
def convert(a):      
    rows, cols = np.where(a.loc[range(1,n+1), range(1,n+1)] > 0)
    edges = zip((rows+1).tolist(), (cols+1).tolist())
    G = nx.Graph()
    G.add_edges_from(edges)

    return G

In [6]:
#parameters
Blocking_Capacity = (int(n/10)*2)
step_size=3

In [7]:
#sets

setV = range(1,n+1)
setS = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] < 1),"IsInfected"].index.tolist()
setP = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] > 0),"IsInfected"].index.tolist()
setRisky= adjacency_df.loc[(adjacency_df.loc[:, "IsRisky"] > 0),"IsRisky"].index.tolist()
setS_diff_index=np.array(setS)-1


In [8]:
clf = DecisionTreeRegressor(random_state=0,min_impurity_decrease=0.01)

In [9]:
#Preprocessing
preprocessed_matrix = adjacency_df.copy(deep=True)



In [10]:
#2 infected 
preprocessed_matrix.loc[setP,setP] = 0




In [11]:
#Sum of all edges lower than threshold (not infected)
index = preprocessed_matrix.loc[(preprocessed_matrix[setV].sum(axis=1)<((preprocessed_matrix.loc[:, "Threshold"]))) & (preprocessed_matrix.loc[:,"IsInfected"]<0.01)  , range(1,n+1)].index
preprocessed_matrix.loc[index,setV] = 0
preprocessed_matrix.loc[setV,index] = 0



In [12]:
#1 neighbor (not infected not risky)
index2 = preprocessed_matrix.loc[((preprocessed_matrix.loc[setV,setV]>0).sum(axis=1) == 1 & (preprocessed_matrix.loc[:,"IsInfected"]<0.01)) & (preprocessed_matrix.loc[:,"IsRisky"]<0.01), range(1,n+1)].index
preprocessed_matrix.loc[index2,setV] = 0
preprocessed_matrix.loc[setV,index2] = 0

In [13]:
#Percentage assignment
preprocessed_matrix.loc[:,setV]=preprocessed_matrix.loc[:,setV].div(preprocessed_matrix.loc[:,"Threshold"],axis=0)
preprocessed_matrix[preprocessed_matrix[setV] >= 1] = 1


In [14]:
nonconnected_nodes=preprocessed_matrix.loc[preprocessed_matrix.loc[:,range(1,n+1)].sum(axis=0)==0,:].index
for i in nonconnected_nodes:
    preprocessed_matrix.at[i,i]=1

In [15]:
#!!!!!!!!! Edges below %50 influence are deleted !!!!!!!!!!!
Graph3=convert(preprocessed_matrix)
preprocessed_matrix[preprocessed_matrix[setV] <= 0.3] = 0

In [16]:
nonconnected_nodes=preprocessed_matrix.loc[preprocessed_matrix.loc[:,range(1,n+1)].sum(axis=0)==0,:].index
for i in nonconnected_nodes:
    preprocessed_matrix.at[i,i]=1

In [17]:
preprocessed_matrix.loc[preprocessed_matrix.loc[:,range(1,n+1)].sum(axis=0)==0,:].index

Int64Index([], dtype='int64')

In [18]:
Graph = convert(preprocessed_matrix)
#Graph.nodes()

In [19]:
found_paths=[]
for starting_nodes in setP:
    for ending_nodes in setRisky:
        found_paths=found_paths+list(nx.all_simple_paths(Graph, starting_nodes, ending_nodes, cutoff=step_size))  

In [20]:
for j in setP:
    found_paths = [[ele for ele in sub if ele != j] for sub in found_paths]

In [21]:
remaining_blocking_capacity=Blocking_Capacity
found_paths_removable=found_paths
already_blocked_nodes=[]
found_paths_np = np.concatenate([np.array(i) for i in found_paths])

In [22]:
nodes_on_paths, total_appearance = np.unique(found_paths_np, return_counts=True)

for i in setP:
    total_appearance=np.delete(total_appearance, np.where(nodes_on_paths[:] == i))
    nodes_on_paths=np.delete(nodes_on_paths, np.where(nodes_on_paths[:] == i))

In [23]:
already_blocked_nodes.append(nodes_on_paths[np.argmax(total_appearance)])
remaining_blocking_capacity -= 1

In [24]:
while (remaining_blocking_capacity>0):
    
    
    found_paths_removable = list(filter(lambda x: already_blocked_nodes[-1] not in x, found_paths_removable))
    
    if (len(found_paths_removable)<=0):
        remaining_risky=np.array(setRisky)

        for i in already_blocked_nodes:
            remaining_risky=np.delete(remaining_risky, np.where(remaining_risky[:] == i))

        if(len(remaining_risky)>0):
            if(len(remaining_risky)>remaining_blocking_capacity):
                chosen=(np.random.choice(remaining_risky,size=remaining_blocking_capacity,replace=False))  
                for items in chosen:
                    already_blocked_nodes.append(items)
            else:
                 for items in remaining_risky:
                    already_blocked_nodes.append(items)   
        
        break
    
    found_paths_removable_np = np.concatenate([np.array(i) for i in found_paths_removable])

    nodes_on_paths, total_appearance = np.unique(found_paths_removable_np, return_counts=True)

    #for i in setP:
        #total_appearance=np.delete(total_appearance, np.where(nodes_on_paths[:] == i))
        #nodes_on_paths=np.delete(nodes_on_paths, np.where(nodes_on_paths[:] == i))

    already_blocked_nodes.append(nodes_on_paths[np.argmax(total_appearance)])
    remaining_blocking_capacity -= 1


In [25]:
blocked_nodes = np.zeros(n)
#blocked_nodes[nodes_on_paths[np.argpartition(total_appearance, -Blocking_Capacity)[-Blocking_Capacity:]]-1]=1


In [26]:
blocked_nodes_df=pd.DataFrame(blocked_nodes).T
blocked_nodes_df.columns=(range(1,(n+1)))
blocked_nodes_df
blocked_nodes_df[already_blocked_nodes]=1

In [27]:
initial_solution_for_tabu=blocked_nodes_df.values.tolist()[0]


In [28]:
#blocked_nodes_df.to_csv('blocked_nodes_df-n'+str(n)+'-m'+str(m)+'.csv',index=False)


In [29]:
def simulate_LTM(removed_nodes,Graph_for_simulation,setP):
    G=Graph_for_simulation.copy()
    for i in range(len(removed_nodes)):
        if(removed_nodes[i]>0):            
            G.remove_node(i+1)
    converted_list=setP.copy()
    while(1):
        #start2=time.time()
        converted_list1 = converted_list[:]
        #print('step:'+str(converted_list))
        for node in G.nodes():
            if (node) not in converted_list: 
                total_weight = 0
                for each in G.neighbors(node):
                    if (each) in converted_list:
                        total_weight = total_weight + adjacency_df.at[node,each]
                if total_weight > adjacency_df.at[node,'Threshold']:
                    converted_list.append(node)
        #end2=time.time()
        #print(end2-start2)           
        if set(converted_list1) == set(converted_list):
            break
    objective_value=adjacency_df.loc[converted_list,'IsRisky'].sum()
    return objective_value,converted_list

In [30]:
def simulate_LTM2(removed_nodes,Graph_for_simulation,setP):
    G=Graph_for_simulation.copy()
    for i in removed_nodes:           
        G.remove_node(i)
    converted_list=setP.copy()   
    while(1):
        converted_list1 = converted_list[:]
        for node in G.nodes():
            if (node) not in converted_list: 
                total_weight = 0
                for each in G.neighbors(node):
                    if (each) in converted_list:
                        total_weight = total_weight + adjacency_df.at[node,each]
                if total_weight > adjacency_df.at[node,'Threshold']:
                    converted_list.append(node)         
        if set(converted_list1) == set(converted_list):
            break
    objective_value=adjacency_df.loc[converted_list,'IsRisky'].sum()
    return objective_value


In [31]:
def NeighborEvaluation(plain_converted_list,incumbent_converted_list,incumbent_objective,Exchanged,Neighbor,Graph_for_simulation,setP):
    if (Exchanged[1] in plain_converted_list):
            for j in range(len(incumbent_converted_list)):
                if (incumbent_converted_list[j]==Exchanged[1]):
                    new_setP=incumbent_converted_list[:j].copy()
                    #print(Exchanged)
                    #print('new_setP:')
                    #print(new_setP)

                    if(incumbent_objective < adjacency_df.loc[new_setP,'IsRisky'].sum()):
                        return ((incumbent_objective+1)*10)
                    return simulate_LTM(Neighbor,Graph_for_simulation,new_setP)[0]
    return incumbent_objective

In [32]:
def NeighborhoodSearch(plain_simulation,IncumbentSolution,TabuList,Graph_for_simulation,setP,tree_list):
    ZeroIndices = []
    OneIndices = []
    
    incumbent_simulation=simulate_LTM(IncumbentSolution,Graph_for_simulation,setP)
    
    for i in range(len(IncumbentSolution)):
        if IncumbentSolution[i] == 0:
            ZeroIndices.append(i)
        else:
            OneIndices.append(i)
    
    AllNeighbor = []
    AllNeighborObjVals = []
    Moves = []
    random_ones= random.sample(range(len(OneIndices)),int((len(OneIndices)*ones[0])))
    random_zeros= random.sample(range(len(ZeroIndices)),int((len(ZeroIndices)*zeros[0])))
    for i in random_ones:
        for j in random_zeros:
    #for i in range(len(OneIndices)):
        #for j in range(len(ZeroIndices)):
            if ((ZeroIndices[j]+1) in setRisky):
                #print('setP')
                #print('____')
                #print(setP)
                if [OneIndices[i],ZeroIndices[j]] not in TabuList:
                    #start=time.time() 
                    Moves.append([OneIndices[i],ZeroIndices[j]])
                    Temp = copy(IncumbentSolution)
                    Temp[ZeroIndices[j]] = 1
                    Temp[OneIndices[i]] = 0
                    Exchanged=[OneIndices[i]+1,ZeroIndices[j]+1]
                    TempObjVal = NeighborEvaluation(plain_simulation,incumbent_simulation[1],incumbent_simulation[0],Exchanged,Temp,Graph_for_simulation,setP)                    
                    #print("TempObjVal:"+str(TempObjVal))
                    AllNeighbor.append(Temp)
                    AllNeighborObjVals.append(TempObjVal)
                    
                    #end=time.time()
                    #print(end-start)
    tree_input_df = pd.DataFrame(AllNeighbor,columns=range(1,n+1))
    tree_target_df = pd.DataFrame(AllNeighborObjVals,columns=["Objective"])
    tree_input_df=tree_input_df.astype("category")
    clf.fit(tree_input_df[setS],tree_target_df)    
    copied_tree = pickle.dumps(clf)
    tree_list.append(pickle.loads(copied_tree))
    
    #print('Tree kuruldu1 '+str(len(tree_target_df)))
    BestIndex = AllNeighborObjVals.index(min(AllNeighborObjVals))
    BestSolution = AllNeighbor[BestIndex]
    BestObjVal = AllNeighborObjVals[BestIndex]
    BestMove = Moves[BestIndex]

    return BestSolution, BestObjVal, BestMove

In [33]:
def NeighborhoodSearch_dm(plain_simulation,IncumbentSolution,TabuList,Graph_for_simulation,setP,tree_list,exponential_smoothing_parameter):
    
    
    ZeroIndices = []
    OneIndices = []
    alpha = exponential_smoothing_parameter
    predicted_out_counter=0
    incumbent_simulation=simulate_LTM(IncumbentSolution,Graph_for_simulation,setP)
    
    for i in range(len(IncumbentSolution)):
        if IncumbentSolution[i] == 0:
            ZeroIndices.append(i)
        else:
            OneIndices.append(i)
    
    AllNeighbor = []
    AllNeighborObjVals = []
    Moves = []
    random_ones= random.sample(range(len(OneIndices)),int((len(OneIndices)*ones[0])))
    random_zeros= random.sample(range(len(ZeroIndices)),int((len(ZeroIndices)*zeros[0])))
    for i in random_ones:
        for j in random_zeros:
    #for i in range(len(OneIndices)):
        #for j in range(len(ZeroIndices)):
            if ((ZeroIndices[j]+1) in setRisky):
                #print('setP')
                #print('____')
                #print(setP)
                if [OneIndices[i],ZeroIndices[j]] not in TabuList:
                    Temp = copy(IncumbentSolution)
                    Temp[ZeroIndices[j]] = 1
                    Temp[OneIndices[i]] = 0
                    ### temp df for tree
                    temp_np=np.array(Temp)
                    prediction_try = temp_np[setS_diff_index]
                    prediction_try=prediction_try.reshape(1,-1)
                    predicted_by_tree=[]
                    prediction=0

                    #for trees in tree_list:
                        #predicted_by_tree.append(trees.predict(prediction_try))
                    #print("prediction started")
                    for t in range(len(tree_list)):
                        prediction+=tree_list[t].predict(prediction_try)*(((1-alpha)**(len(tree_list)-t-1)))*(alpha)
                    #print("prediction ended")
                    #print("___________________________________________")
                    #print("Tree prediction: " + str(prediction))
                    if prediction < incumbent_simulation[0]*(1.0):
                        Moves.append([OneIndices[i],ZeroIndices[j]])
                        Exchanged=[OneIndices[i]+1,ZeroIndices[j]+1]
                        TempObjVal = NeighborEvaluation(plain_simulation,incumbent_simulation[1],incumbent_simulation[0],Exchanged,Temp,Graph_for_simulation,setP)                    
                        #print("TempObjVal:"+str(TempObjVal))
                        AllNeighbor.append(Temp)
                        AllNeighborObjVals.append(TempObjVal)
                        #print("Simulation: " + str(TempObjVal))
                        #print("Difference: " + str(TempObjVal-prediction))
                        #end=time.time()
                        #print(end-start)
                    else:
                        predicted_out_counter += 1
                        if (predicted_out_counter == 3):
                            predicted_out_counter = 0
                            Moves.append([OneIndices[i],ZeroIndices[j]])
                            Exchanged=[OneIndices[i]+1,ZeroIndices[j]+1]
                            TempObjVal = NeighborEvaluation(plain_simulation,incumbent_simulation[1],incumbent_simulation[0],Exchanged,Temp,Graph_for_simulation,setP)                    
                            #print("TempObjVal:"+str(TempObjVal))
                            AllNeighbor.append(Temp)
                            AllNeighborObjVals.append(TempObjVal)
                            #print("Simulation of Pred_Out: " + str(TempObjVal))
                            #print("Incumbent Threshold: " + str(incumbent_simulation[0]*(1.1)))
                            #print("Tree prediction: " + str(prediction))
                            
    tree_input_df = pd.DataFrame(AllNeighbor,columns=range(1,n+1))
    tree_target_df = pd.DataFrame(AllNeighborObjVals,columns=["Objective"])
    tree_input_df=tree_input_df.astype("category")    
    clf.fit(tree_input_df[setS],tree_target_df)    
    copied_tree = pickle.dumps(clf)
    tree_list.append(pickle.loads(copied_tree))
    #print('Tree kuruldu2 '+str(len(tree_target_df)))
    BestIndex = AllNeighborObjVals.index(min(AllNeighborObjVals))
    BestSolution = AllNeighbor[BestIndex]
    BestObjVal = AllNeighborObjVals[BestIndex]
    BestMove = Moves[BestIndex]

    return BestSolution, BestObjVal, BestMove

In [34]:
def TabuSearchBinarySwap(NumberOfOnes,SpinningLimit,MaxIter,TabuTenure,Graph_for_simulation,setP,initial_solution_for_tabu,plain_simulation,initial_tree,exponential_smoothing_parameter):
    
    tree_list = []
    
    
    copied_tree = pickle.dumps(clf)
    tree_list.append(pickle.loads(copied_tree))
    
    random.seed(123)
    
    
    InitialSolution = initial_solution_for_tabu.copy() 

    IncumbentSolution = InitialSolution
    IncumbentObjVal = simulate_LTM(IncumbentSolution,Graph_for_simulation,setP)[0]
    #print("tabuda çıktım")
    BestSolution = IncumbentSolution
    BestObjVal = IncumbentObjVal
    Trajectory = [IncumbentObjVal]
    BestTrajectory = [BestObjVal]
    
    TabuList = []
    #for i in setS:
     #   for j in setP:
     #       TabuList = [j,i]
    
    SpinningIndex = 0
    Iteration = 0
    
    start_CPU=time.time()
    CPU_time=0
    
    while ((Iteration < MaxIter) and CPU_time<=7200):
        if Iteration < 3 : 
        #start=time.time()        
            Out = NeighborhoodSearch(plain_simulation,IncumbentSolution,TabuList,Graph_for_simulation,setP,tree_list) 
        else:
            Out= NeighborhoodSearch_dm(plain_simulation,IncumbentSolution,TabuList,Graph_for_simulation,setP,tree_list,exponential_smoothing_parameter) 
            
        #print(Out[1])
        #end=time.time()
        #print(end-start)
        BestNeighbor = Out[0]
        BestNeighborObjVal = Out[1]
        BestNeighborMove = Out[2]
        
        if BestObjVal <= IncumbentObjVal:
            
            IncumbentSolution = BestNeighbor
            IncumbentObjVal = BestNeighborObjVal
            Trajectory.append(IncumbentObjVal)
            if IncumbentObjVal < BestObjVal:
                BestObjVal = IncumbentObjVal
                BestSolution = IncumbentSolution
                SpinningIndex = 0
            else:
                SpinningIndex += 1
            BestTrajectory.append(BestObjVal)
            TabuList.append(BestNeighborMove)
            
        else:
            
            Trajectory.append(IncumbentObjVal)
            BestTrajectory.append(BestObjVal)
            SpinningIndex += 1
            
            
        if len(TabuList) > TabuTenure:
            TabuList.pop(0)
            
        if SpinningIndex > SpinningLimit:
            
            Iteration = MaxIter
            
        else:
        
            Iteration += 1
    
        end_CPU=time.time()
        CPU_time=end_CPU-start_CPU
    return BestSolution, BestObjVal, BestTrajectory, Trajectory, TabuList



In [35]:
total_rows=n*2
all_columns=setS+['Objective']
df_for_dm = pd.DataFrame(np.zeros((total_rows,(n-len(setP)+1))),columns=all_columns,index=range(1,total_rows+1))
for i in range(1,total_rows+1):
    solution_vector=np.random.choice(setS,Blocking_Capacity,replace=False)
    df_for_dm.at[i,'Objective']=simulate_LTM2(solution_vector,Graph,setP)
    df_for_dm.loc[i,solution_vector]=1
InputVars=df_for_dm.iloc[:,:-1]
Target=df_for_dm.iloc[:,-1]
InputVars=InputVars.astype("category")    
clf.fit(InputVars,Target)


DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.01, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')

In [36]:
#tree.plot_tree(clf)
#plt.savefig('Tree_2.pdf')
#plt.show()


In [37]:
SpinningLimit = 5
MaxIter = n*5
TabuTenure = 20
exponential_smoothing_parameter=0.7
Graph2 = convert(adjacency_df)
bosliste=np.zeros(n).tolist()
plain_simulation=simulate_LTM(bosliste,Graph2,setP)[1]
start1=time.time()
Tabu_Results = TabuSearchBinarySwap(Blocking_Capacity,SpinningLimit,MaxIter,TabuTenure,Graph2,setP,initial_solution_for_tabu,plain_simulation,clf,exponential_smoothing_parameter)
end1=time.time()
print(end1-start1)

3.111018657684326


In [38]:
print('TABU ONLY RISKY OBJECTIVE VALUE: '+str(Tabu_Results[1]))

TABU ONLY RISKY OBJECTIVE VALUE: 10.0


In [39]:
print('TABU ONLY RISKY TIME: '+str(end1-start1))

TABU ONLY RISKY TIME: 3.111018657684326
